# <center>PySpark Project: HPI_AT_BDL_ZIP3</center>

| Course | Instructor | Full Name | Date       | Term   |
|--------|------------|-----------|------------|--------|
| ALY6110 | Prof Andy Chan, Faculty Lecturer | Abhilash Dikshit | Jun 24, 2023 | Spring |


In [ ]:
#!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

In [ ]:
# Create SparkSession
spark = SparkSession.builder.appName("ZIP Code Housing Price Analysis").getOrCreate()

In [ ]:
# Load the dataset
df = spark.read.format("csv").option("header", "true").load("/Users/abidikshit/GitProjects/Datasets/HPI_AT_BDL_ZIP3.csv")

In [ ]:
# Print the schema of the DataFrame
df.printSchema()

In [ ]:
# Drop columns _c6
df = df.drop("_c6")

# Print the updated schema of the DataFrame
df.printSchema()

In [ ]:
from pyspark.sql.functions import col

# Convert columns to appropriate data types
df = df.withColumn("Year", col("Year").cast("integer"))
df = df.withColumn("Annual Change (%)", col("Annual Change (%)").cast("double"))
df = df.withColumn("HPI", col("HPI").cast("double"))
df = df.withColumn("HPI with 1990 base", col("HPI with 1990 base").cast("double"))
df = df.withColumn("HPI with 2000 base", col("HPI with 2000 base").cast("double"))

In [ ]:
# Data Exploration
df.printSchema()
df.show()

In [ ]:
# Descriptive Statistics
df.describe().show()

In [ ]:
# Time Series Analysis
time_series_data = df.select("Year", "HPI").orderBy("Year")
years = [row["Year"] for row in time_series_data.collect()]
hpi_values = [row["HPI"] for row in time_series_data.collect()]
plt.plot(years, hpi_values)
plt.xlabel("Year")
plt.ylabel("HPI")
plt.title("Housing Price Index Over Time")
plt.show()

In [ ]:
# Group by year and calculate average HPI for each year
seasonal_data = df.groupBy("Year").agg(F.avg("HPI").alias("Average HPI"))

# Order the data by year
seasonal_data = seasonal_data.orderBy("Year")

# Extract year and average HPI values
years = [row["Year"] for row in seasonal_data.collect()]
hpi_values = [row["Average HPI"] for row in seasonal_data.collect()]

# Plot the seasonality pattern
plt.plot(years, hpi_values)
plt.xlabel("Year")
plt.ylabel("Average HPI")
plt.title("Seasonality Patterns in HPI")
plt.show()


In [ ]:
# Correlation Analysis
correlation = df.stat.corr("Annual Change (%)", "HPI with 2000 base")
print("Correlation between Annual Change (%) and HPI with 2000 base:", correlation)

In [ ]:
# Close SparkSession
spark.stop()